<h1><b> 🗣️ Traductor Subtítols VTT amb el model AINA 🗣️ </h1></b>

---



```
# Copyright (c) Huilin Ni, https://www.linkedin.com/in/huilin-ni-71511b215/, [2024]
```



## **1. Preparació de l'entorn de treball**

### 1.1. Instal·lació dels paquets

In [ ]:
!pip install ctranslate2 pyonmttok
# !pip install alive-progress
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/36.7 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 26.8 MB/s eta 0:00:00


### 1.2. Mount per carregar les carpetes del Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### 1.3. Imports dels paquets que es fan servir

In [ ]:
import os
import torch
import ctranslate2
import pyonmttok

# La funció "snapshot_download" ens permet descarregar models de Hugging Face Model Hub
from huggingface_hub import snapshot_download
# from alive_progress import alive_bar
from tqdm import tqdm

### 1.4. Executar a la GPU en cas que estigui disponible

In [ ]:
# Comprovem si està disponible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## **2. Carreguem el model d'AINA**

In [16]:
# Descarreguem el model català-espanyol
model_ca_es = snapshot_download(repo_id="projecte-aina/aina-translator-ca-es", revision="main")

# Descarreguem el model català-anglès
model_ca_en = snapshot_download(repo_id="projecte-aina/aina-translator-ca-en", revision="main")

# Descarreguem el model espanyol-català
model_es_ca = snapshot_download(repo_id="projecte-aina/aina-translator-es-ca", revision="main")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
# Creem el tokenitzador pels models
tokenizer_ca_es = pyonmttok.Tokenizer(mode="none", sp_model_path = model_ca_es + "/spm.model")
tokenizer_ca_en = pyonmttok.Tokenizer(mode="none", sp_model_path = model_ca_en + "/spm.model")
tokenizer_es_ca = pyonmttok.Tokenizer(mode="none", sp_model_path = model_es_ca + "/spm.model")

# Carreguem els models de traducció
translator_ca_es = ctranslate2.Translator(model_ca_es)
translator_ca_en = ctranslate2.Translator(model_ca_en)
translator_es_ca = ctranslate2.Translator(model_es_ca)

## **3. Definició de funcions**

In [14]:
# Funció per trobar els fitxers VTT d'una path
def collect_vtt_files(folder_path):
    vtt_files = []
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith("SubtCA-rev.vtt") or file_name.endswith("SubtES-rev.vtt"):
                vtt_files.append((root, file_name))
    return vtt_files


# Funció que tradueix els fitxers vtt
def translate_vtt_file(input_file, output_file, language):

    # Determinem a quins idiomes es poden fer les traduccions depenent de l'input
    if input_file.endswith("SubtCA-rev.vtt"):
        available_languages = ["es", "en"]
    elif input_file.endswith("SubtES-rev.vtt"):
        available_languages = ["ca"]
    else:
        raise ValueError("Unsupported file format")

    if language not in available_languages:
        raise ValueError(f"Unsupported target language. Available languages: {', '.join(available_languages)}")

    # Carreguem el traductor i el tokenizer segons l'idioma de target
    if input_file.endswith("SubtCA-rev.vtt"):
        if language == "es":
            translator = translator_ca_es
            tokenizer = tokenizer_ca_es
        elif language == "en":
            translator = translator_ca_en
            tokenizer = tokenizer_ca_en
    elif input_file.endswith("SubtES-rev.vtt"):
        if language == "ca":
            translator = translator_es_ca
            tokenizer = tokenizer_es_ca

    # Llegim el contingut de l'arxiu VTT
    with open(input_file, 'r', encoding='utf-8') as f:
        vtt_content = f.read()

    # Dividim el contingut en línies
    lines = vtt_content.split('\n')
    # print(lines)

    # Traduïm cada línea (excloent la línia del temps) i els espais buits
    translated_lines = []
    input_filename = os.path.basename(input_file)

    # Inicialitzem la barra del progrés
    progress_bar = tqdm(total=len(lines), desc=f"Translating {input_filename} to {language.upper()}", position=0, leave=True)

    for line in lines:
        if '-->' in line:  # Conservem la línia de temps sense traduir
            translated_lines.append(line)
        elif line == '':  # línies buides, els salts de línia
            translated_lines.append(line)
        else:
            tokenized = tokenizer.tokenize(line)
            translated = translator.translate_batch([tokenized[0]])
            translated_text = tokenizer.detokenize(translated[0][0]['tokens'])
            # print(f"\n {translated_text}")
            translated_lines.append(translated_text)
        progress_bar.update(1)

    # print(translated_lines)

    # Escribim les línies traduïdes a l'arxiu d'output
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(translated_lines))


# Funció que s'encarrega de reanomenar el fitxer d'output i crida a la funció que tradueix
def translate_and_save_vtt_file(vtt_files, target_language):
    for root, file_name in vtt_files:
        input_file = os.path.join(root, file_name)
        # print(input_file)
        input_filename = os.path.basename(input_file)

        # Iterem per fer la traducció als idiomes especificats
        for language in target_language:
            output_file = input_file.replace("CA", f"{language.upper()}")
            # print(output_file)
            translate_vtt_file(input_file, output_file, language)
            output_filename = os.path.basename(output_file)

            print(f"\n File '{input_filename}' has been translated to '{language.upper()} and has been saved as '{output_filename}' in {output_file}. \n")

    print("\n Translation process finished. \n")

## **4. Declaració dels paràmetres globals**

In [ ]:
# Path del directori
folder_path = "/content/drive/MyDrive/2023-2024TopRosiesTalent3aed/OperacioInternacionalitzacio/videosTRT3aed/01_Introduction and ethics of AI/Chapter 1 - Introduction to AI/Lesson 1. The Concept of AI/1Darmouth"

# Idioma al que es vol traduir. Opcions: "es" - espanyol, "en" - anglès, "ca" - català
target_language = ["es", "en"]

## **5. Execució del procés**

In [15]:
vtt_files = collect_vtt_files(folder_path)
# print(vtt_files)
translate_and_save_vtt_file(vtt_files, target_language)

Translating 1DarmouthSubtCA-rev.vtt to ES:   0%|          | 0/82 [00:00<?, ?it/s]<ipython-input-14-59afc3f70a2f>:61: DeprecationWarning: Reading the TranslationResult object as a list of dictionaries is deprecated and will be removed in a future version. Please use the object attributes as described in the documentation: https://opennmt.net/CTranslate2/python/ctranslate2.TranslationResult.html
  translated_text = tokenizer.detokenize(translated[0][0]['tokens'])
Translating 1DarmouthSubtCA-rev.vtt to ES: 100%|██████████| 82/82 [00:46<00:00,  1.78it/s]



 File '1DarmouthSubtCA-rev.vtt' has been translated to 'ES and has been saved as '1DarmouthSubtES-rev.vtt' in /content/drive/MyDrive/2023-2024TopRosiesTalent3aed/OperacioInternacionalitzacio/videosTRT3aed/01_Introduction and ethics of AI/Chapter 1 - Introduction to AI/Lesson 1. The Concept of AI/1Darmouth/1DarmouthVTTrevisats/1DarmouthSubtES-rev.vtt. 



Translating 1DarmouthSubtCA-rev.vtt to EN: 100%|██████████| 82/82 [00:52<00:00,  1.56it/s]


 File '1DarmouthSubtCA-rev.vtt' has been translated to 'EN and has been saved as '1DarmouthSubtEN-rev.vtt' in /content/drive/MyDrive/2023-2024TopRosiesTalent3aed/OperacioInternacionalitzacio/videosTRT3aed/01_Introduction and ethics of AI/Chapter 1 - Introduction to AI/Lesson 1. The Concept of AI/1Darmouth/1DarmouthVTTrevisats/1DarmouthSubtEN-rev.vtt. 


 Translation process finished. 

